## Char-RNN

In this tutorial, we will build a char-rnn model for natural language generation. The training text is tokenized as a sequence of characters. After training, the model is able to output the probability distribution over the alphabet, therefore "predicting" the next character. By iterating this process, one can generate text snippets.

Char-RNN processes text sequences of arbitrary length, and the loss function makes use of ordinary Scala control-flow features during the training phase. Therefore it is an instance of dynamic neural network.

This implementation of Char-RNN is inspired by Andrej Karpathy's execellent blog post [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [Python/numpy implementation](https://gist.github.com/karpathy/d4dee566867f8291f086).

## Importing dependencies

In [1]:
import $ivy.`org.nd4j:nd4j-native-platform:0.8.0`
import $ivy.`com.thoughtworks.deeplearning::plugins-cumulativedoublelayers:2.0.0-RC6`
import $ivy.`com.thoughtworks.deeplearning::plugins-doubletraining:2.0.0-RC6`
import $ivy.`com.thoughtworks.deeplearning::plugins-cumulativeindarraylayers:2.0.0-RC6`
import $ivy.`com.thoughtworks.deeplearning::plugins-indarrayweights:2.0.0-RC6`
import $ivy.`com.thoughtworks.deeplearning::plugins-indarrayliterals:2.0.0-RC6`
import $ivy.`com.thoughtworks.deeplearning::plugins-logging:2.0.0-RC6`

import scala.math
import collection.immutable.IndexedSeq
import scala.io.Source
import scala.concurrent.ExecutionContext.Implicits.global
import scalaz.concurrent.Task
import scalaz.std.iterable._
import scalaz.syntax.all._
import com.thoughtworks.future._
import scala.concurrent.Await
import scala.concurrent.duration.Duration
import org.nd4j.linalg.factory.Nd4j
import org.nd4j.linalg.api.ndarray.INDArray
import org.nd4j.linalg.ops.transforms.Transforms
import org.nd4j.linalg.api.ops.impl.indexaccum.IMax
import com.thoughtworks.deeplearning.plugins.DoubleLiterals
import com.thoughtworks.deeplearning.plugins.INDArrayLiterals
import com.thoughtworks.deeplearning.plugins.CumulativeDoubleLayers
import com.thoughtworks.deeplearning.plugins.DoubleTraining
import com.thoughtworks.deeplearning.plugins.CumulativeINDArrayLayers
import com.thoughtworks.deeplearning.plugins.INDArrayWeights
import com.thoughtworks.deeplearning.plugins.Operators
import com.thoughtworks.deeplearning.plugins.Logging
import com.thoughtworks.feature.Factory

import $ivy.$                                    

import $ivy.$                                                                        

import $ivy.$                                                                

import $ivy.$                                                                          

import $ivy.$                                                                 

import $ivy.$                                                                  

import $ivy.$                                                         


import scala.math

import collection.immutable.IndexedSeq

import scala.io.Source

import scala.concurrent.ExecutionContext.Implicits.global

import scalaz.concurrent.Task

import scalaz.std.iterable._

import scalaz.syntax.all._

import com.thoughtworks.future._

import scala.concurrent.Await

import scala.concurrent.duration.Duration

import org.nd4j.linalg.factory.Nd4j

import org.nd4j.linalg.api.ndarray.INDArray

import org.nd4j.linalg.ops.transforms.T

## Preparing the corpus, setting up plugins & parameters

In [2]:
val data = "DeepLearning.scala is a framework for type-safe neural networks."
val dataSize = data.size

val ixToChar = data.toSet.toArray
val charToIx = (for (i <- ixToChar.indices) yield (ixToChar(i), i)).toMap
val vocabSize = ixToChar.size

def oneOfK(c: Char) = Nd4j.zeros(vocabSize, 1).putScalar(charToIx(c), 1)

data: String = "DeepLearning.scala is a framework for type-safe neural networks."
dataSize: Int = 64
ixToChar: Array[Char] = Array(
  'e',
  's',
  'n',
  '.',
  'y',
  't',
  'u',
  'f',
  'a',
  'm',
  'i',
...
charToIx: Map[Char, Int] = Map(
  'e' -> 0,
  's' -> 1,
  'n' -> 2,
  '.' -> 3,
  'y' -> 4,
  't' -> 5,
  'u' -> 6,
  'f' -> 7,
  'a' -> 8,
  'm' -> 9,
  'i' -> 10,
...
vocabSize: Int = 23
defined function oneOfK

In [3]:
trait LearningRate extends INDArrayWeights {
    val learningRate: Double
    
    trait INDArrayOptimizerApi extends super.INDArrayOptimizerApi { this: INDArrayOptimizer =>
      override def delta: INDArray = super.delta mul learningRate
    }
    override type INDArrayOptimizer <: INDArrayOptimizerApi with Optimizer
  }

trait Adagrad extends INDArrayWeights {
    val eps: Double
    
    trait INDArrayWeightApi extends super.INDArrayWeightApi { this: INDArrayWeight =>
      var cache: Option[INDArray] = None
    }

    override type INDArrayWeight <: INDArrayWeightApi with Weight

    trait INDArrayOptimizerApi extends super.INDArrayOptimizerApi { this: INDArrayOptimizer =>
      private lazy val deltaLazy: INDArray = {
        import org.nd4s.Implicits._
        import weight._
        val delta0 = super.delta
        cache = Some(cache.getOrElse(Nd4j.zeros(delta0.shape: _*)) + delta0 * delta0)
        delta0 / (Transforms.sqrt(cache.get) + eps)
      }
      override def delta = deltaLazy
    }
    override type INDArrayOptimizer <: INDArrayOptimizerApi with Optimizer
  }

defined trait LearningRate
defined trait Adagrad

In [5]:
interp.load("""
  val hyperparameters = Factory[Adagrad with LearningRate with DoubleTraining with CumulativeDoubleLayers with CumulativeINDArrayLayers with Operators with INDArrayLiterals with DoubleLiterals with Logging].newInstance(learningRate = 0.1, eps=1e-8)
""")

In [6]:
import hyperparameters.INDArrayWeight
import hyperparameters.DoubleLayer
import hyperparameters.INDArrayLayer
import hyperparameters.implicits._

import hyperparameters.INDArrayWeight

import hyperparameters.DoubleLayer

import hyperparameters.INDArrayLayer

import hyperparameters.implicits._

In [7]:
val hiddenSize = 100
val seqLength = 25

val wxh = {
    import org.nd4s.Implicits._
    INDArrayWeight(Nd4j.randn(hiddenSize, vocabSize) * 0.01)
}

val whh = {
    import org.nd4s.Implicits._
    INDArrayWeight(Nd4j.randn(hiddenSize, hiddenSize) * 0.01)
}

val why = {
    import org.nd4s.Implicits._
    INDArrayWeight(Nd4j.randn(vocabSize, hiddenSize) * 0.01)
}

val bh = INDArrayWeight(Nd4j.zeros(hiddenSize, 1))
val by = INDArrayWeight(Nd4j.zeros(vocabSize, 1))

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


hiddenSize: Int = 100
seqLength: Int = 25
wxh: Object with hyperparameters.INDArrayWeightApi with hyperparameters.WeightApi with hyperparameters.WeightApi with hyperparameters.INDArrayWeightApi = $sess.cmd4Wrapper$Helper$Anonymous$macro$1$1$Anonymous$macro$15$1@2407052c
whh: Object with hyperparameters.INDArrayWeightApi with hyperparameters.WeightApi with hyperparameters.WeightApi with hyperparameters.INDArrayWeightApi = $sess.cmd4Wrapper$Helper$Anonymous$macro$1$1$Anonymous$macro$15$1@4fb3ebd6
why: Object with hyperparameters.INDArrayWeightApi with hyperparameters.WeightApi with hyperparameters.WeightApi with hyperparameters.INDArrayWeightApi = $sess.cmd4Wrapper$Helper$Anonymous$macro$1$1$Anonymous$macro$15$1@7c2bdb6a
bh: Object with hyperparameters.INDArrayWeightApi with hyperparameters.WeightApi with hyperparameters.WeightApi with hyperparameters.INDArrayWeightApi = $sess.cmd4Wrapper$Helper$Anonymous$macro$1$1$Anonymous$macro$15$1@7f8c8adb
by: Object with hyperparameters.INDArrayWei

## Implementing the neural network

In [8]:
def tanh(x: INDArrayLayer): INDArrayLayer = {
  val exp_x = hyperparameters.exp(x)
  val exp_nx = hyperparameters.exp(-x)
  (exp_x - exp_nx) / (exp_x + exp_nx)
}

defined function tanh

In [9]:
def charRNN(x: INDArray, y: INDArray, hprev: INDArrayLayer): (DoubleLayer, INDArrayLayer, INDArrayLayer) = {
    val hnext = tanh(wxh.dot(x) + whh.dot(hprev) + bh)
    val yraw = why.dot(hnext) + by
    val yraw_exp = hyperparameters.exp(yraw)
    val prob = yraw_exp / yraw_exp.sum
    val loss = -hyperparameters.log((prob * y).sum)
    (loss, prob, hnext)
}

defined function charRNN

In [10]:
val batches = data.zip(data.tail).grouped(seqLength).toVector

type WithHiddenLayer[A] = (A, INDArrayLayer)
type Batch = IndexedSeq[(Char, Char)]
type Losses = Vector[Double]

def singleBatch(batch: WithHiddenLayer[Batch]): WithHiddenLayer[DoubleLayer] = {
  batch match {
    case (batchseq, hprev) => batchseq.foldLeft((DoubleLayer(0.0.forward), hprev)) {
      (bstate: WithHiddenLayer[DoubleLayer], xy: (Char, Char)) =>
        (bstate, xy) match {
          case ((tot, localhprev), (x, y)) => {
            charRNN(oneOfK(x), oneOfK(y), localhprev) match {
              case (localloss, _, localhnext) => {
                (tot + localloss, localhnext)
              }
            }
          }
        }
    }
  }
}

def initH = INDArrayLayer(Nd4j.zeros(hiddenSize, 1).forward)

def singleRound(initprevloss: Losses): Future[Losses] =
  (batches.foldLeftM((initprevloss, initH)) {
    (bstate: WithHiddenLayer[Losses], batch: Batch) =>
      bstate match {
        case (prevloss, hprev) => singleBatch(batch, hprev) match {
          case (bloss, hnext) => bloss.train.map {
            (blossval: Double) => {
                val nloss = prevloss.last * 0.999 + blossval * 0.001
                publish.md(nloss.toString)
                (prevloss :+ prevloss.last * 0.999 + blossval * 0.001, hnext)
            }
          }
        }
      }
  }).map {
    (fstate: WithHiddenLayer[Losses]) =>
      fstate match {
        case (floss, _) => floss
      }
  }

def allRounds: Future[Losses] = (0 until 1024).foldLeftM(Vector(-math.log(1.0 / vocabSize) * seqLength)) {
  (ploss: Losses, round: Int) => {
      publish.md("Round: " + round.toString)
      singleRound(ploss)
  }
}

batches: Vector[IndexedSeq[(Char, Char)]] = Vector(
  Vector(
    ('D', 'e'),
    ('e', 'e'),
    ('e', 'p'),
    ('p', 'L'),
    ('L', 'e'),
    ('e', 'a'),
    ('a', 'r'),
    ('r', 'n'),
    ('n', 'i'),
    ('i', 'n'),
...
defined type WithHiddenLayer
defined type Batch
defined type Losses
defined function singleBatch
defined function initH
defined function singleRound
defined function allRounds

## Training the model and using it to generate text

In [11]:
def unsafePerformFuture[A](f: Future[A]): A = Await.result(f.asScala, Duration.Inf)

val losses = unsafePerformFuture(allRounds)

Round: 0

78.38735550768736

78.50146961331119

78.70391376568662

Round: 1

...

Round: 1023

9.953100486289353

9.944511959567617

9.935501253819709

defined function unsafePerformFuture
losses: Losses = Vector(
  78.38735539822875,
  78.38735550768736,
  78.50146961331119,
  78.70391376568662,
  79.08358185150873,
  79.29432202854235,
  79.41762726394649,
  79.60124919106794,
  79.70259113849347,
  79.7225872169379,
  79.84444848397035,
...

In [12]:
def genIdx(v: INDArray): Int = Nd4j.getExecutioner().execAndReturn(new IMax(v)).getFinalResult()

def generate(seed: Char, n: Int): Future[String] = ((0 until n).foldLeftM((seed.toString, initH)) {
    (st: (String, INDArrayLayer), i: Int) =>
        st match {
            case (tot, hprev) => {
                val x = oneOfK(tot.last)
                charRNN(x, x, hprev) match {
                    case (_, prob, hnext) =>
                        prob.predict.flatMap { (probv: INDArray) =>
                            val nidx = genIdx(probv)
                            val nc = ixToChar(nidx)
                            Future.now(tot + nc.toString, hnext)
                        }
                }
            }
        }
}).map { (st: (String, INDArrayLayer)) =>
  st match {
    case (r, _) => r
  }
}

defined function genIdx
defined function generate

In [13]:
unsafePerformFuture(generate('D', 128))

res12: String = "DeepLearnila is a framework for type-safe neural networks.forural networks.fnsural networks.forurae networks.fosural networks.for"